In [1]:
from keras.models import model_from_json
import os
import cv2
import glob
import h5py
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import scipy.io as io
from PIL import Image
import numpy as np

In [2]:
def load_model():
    
    json_file = open('models/Model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("weights/model_A_weights.h5")
    return loaded_model


def create_img(path):
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225


    im = np.expand_dims(im,axis  = 0)
    return im

In [3]:
root = 'data'

In [4]:
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

182


In [13]:
path_sets = [part_A_train]

In [14]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

300


In [15]:
model = load_model()
name = []
y_true = []
y_pred = []


for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    num = np.sum(model.predict(img))
    y_pred.append(np.sum(num))

    
data = pd.DataFrame({'name': name,'y_pred': y_pred,'y_true': y_true})
data.to_csv('CSV/A_on_A_train.csv', sep=',')

<ipython-input-15-41b9a64cc87c>:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )


In [19]:
path_sets = [part_B_train]

In [20]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

400


In [21]:
model = load_model()
name = []
y_true = []
y_pred = []


for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    num = np.sum(model.predict(img))
    y_pred.append(np.sum(num))

    
data = pd.DataFrame({'name': name,'y_pred': y_pred,'y_true': y_true})
data.to_csv('CSV/B_on_B_train.csv', sep=',')

<ipython-input-21-c4b1267121f6>:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )


# MAE for train 

In [11]:
data = pd.read_csv('CSV/A_on_A_train.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [12]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [13]:
print("MAE : " , ans )

MAE :  22.619478651682535


In [15]:
data = pd.read_csv('CSV/B_on_B_train.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [16]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [17]:
print("MAE : " , ans )

MAE :  20.759766745567322


# MAE for test

In [19]:
data = pd.read_csv('CSV/A_on_A_test.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [20]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [21]:
print("MAE : " , ans )

MAE :  65.92782475398137


In [23]:
data = pd.read_csv('CSV/B_on_B_test.csv' , sep='\t')
y_true = data['y_true']
y_pred = data['y_pred']

In [24]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [25]:
print("MAE : " , ans )

MAE :  11.001472832281378


In [26]:
ans = mean_squared_error(np.array(y_true),np.array(y_pred))
ans

266.23280174884786